# Flow Metrics

This workbook generates reports on flow metric from a given GitHub repo using the Github API.

Author: [Brian McIlwain](mailto:brian@poq.gg)

## Setup

This gets data from the [Github GraphQL API](https://docs.github.com/en/graphql/overview/about-the-graphql-api) to create reports for flow metrics

### Imports, utils, & globals

In [1]:
from datetime import date, datetime, timezone
from dateutil.relativedelta import relativedelta, SU
from functools import reduce
import pandas as pd
from gql.transport.aiohttp import AIOHTTPTransport
from gql import gql, Client
from os import environ
from dotenv import load_dotenv
import numpy as np

load_dotenv()
API_SECRET_TOKEN = environ.get('API_SECRET_TOKEN')

if not API_SECRET_TOKEN:
    raise Exception(
        'API_SECRET_TOKEN is not defined as an environmental variable')

# Config for repo
API_URL = "https://api.github.com/graphql"
REPO_OWNER = "weiks"
REPO_NAME = "esports-backend"
MAX_WEEK_ISSUES = 100  # API limit, shouldn't be hit
MAX_PAGINATION_LIMIT = 100  # Set by API, I wish it was larger
MAX_LABEL_LIMIT = 20  # Max labels per PR

print(f"Repo: {REPO_OWNER}/{REPO_NAME}\n")

# Prep to run GQL calls
headers = {"Authorization": f"Bearer {API_SECRET_TOKEN}"}
transport = AIOHTTPTransport(url=API_URL, headers=headers)


async def runQuery(query, variable_values=None):
    # Create a GraphQL client using the defined transport
    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:
        # Execute the query on the transport
        result = await session.execute(query, variable_values)
        return result


def get_previous_sunday(working_date=date.today()):
    last_sunday = working_date + relativedelta(weekday=SU(-1))
    return last_sunday.strftime("%Y-%m-%d")


def unwrap_name(d):
    return d['name']


def unwrap_login(d):
    return d['login']


def clean_issue_data(issue):
    issue['author'] = issue['author']['login']
    issue['labels'] = set(map(unwrap_name, issue['labels']['nodes']))
    issue['assignees'] = set(map(unwrap_login, issue['assignees']['nodes']))
    # Calculate durations
    issue['createdAt'] = pd.Timestamp(
        pd.to_datetime(issue['createdAt'], errors='coerce'))
    issue['closedAt'] = pd.Timestamp(
        pd.to_datetime(issue['closedAt'], errors='coerce'))
    issue['daysFromOpen'] = ((datetime.now(timezone.utc)) - issue['createdAt']) / \
        np.timedelta64(1, 'D')
    issue['daysToClose'] = (issue['closedAt'] - issue['createdAt']) / \
        np.timedelta64(1, 'D')

    return issue


def clean_PR_data(pr):
    pr['author'] = pr['author']['login']
    pr['assignees'] = set(map(unwrap_login, pr['assignees']['nodes']))
    pr['assignees'].add(pr['author'])  # Author is an assignee by default
    pr['issues'] = list(map(clean_issue_data, pr['issues']['nodes']))
    pr['labels'] = set(map(unwrap_name, pr['labels']['nodes']))
    pr['labels'].discard('trigger-ci')  # Ignore trigger-ci
    # Calculate durations
    pr['createdAt'] = pd.Timestamp(
        pd.to_datetime(pr['createdAt'], errors='coerce'))
    pr['closedAt'] = pd.Timestamp(
        pd.to_datetime(pr['closedAt'], errors='coerce'))
    pr['hoursFromOpen'] = (datetime.now(timezone.utc) - pr['createdAt']) / \
        np.timedelta64(1, 'h')
    pr['hoursToClose'] = (pr['closedAt'] - pr['createdAt']) / \
        np.timedelta64(1, 'h')

    # Apply parent issue labels to the PR
    if pr['issues']:
        issues_labels = list(map(lambda issue: issue['labels'], pr['issues']))
        flat_issues_labels = reduce(lambda a, b: a.union(b), issues_labels)
        pr['labels'] = pr['labels'].union(flat_issues_labels)

    return pr


Repo: weiks/esports-backend



## Get PRs Data

In [2]:
async def get_PR_data():
    cursor = ""
    prs = []
    print('Fetching PRs, this can take a while...')

    while cursor != None:
        overviewQuery = gql(
            f"""
            query getRepoData {{
              repository(owner: "{REPO_OWNER}", name: "{REPO_NAME}") {{
                pullRequests(
                  first: {MAX_PAGINATION_LIMIT},
                  orderBy: {{ field: UPDATED_AT, direction: DESC }}
                  { f'after: "{cursor}"' if cursor else '' }
                ) {{
                  pageInfo {{
                    hasNextPage
                    endCursor
                  }}
                  nodes {{
                    title
                    author {{
                      login
                    }}
                    assignees(first: 10) {{
                      nodes {{
                        login
                      }}
                    }}
                    number
                    url
                    state
                    closedAt
                    createdAt
                    updatedAt
                    labels(first: {MAX_LABEL_LIMIT}) {{
                      nodes {{
                        name
                      }}
                    }}
                    issues: closingIssuesReferences(first: {MAX_LABEL_LIMIT}) {{
                      nodes {{
                        assignees(first: 10) {{
                          nodes {{
                            login
                          }}
                        }}
                        labels(first: {MAX_LABEL_LIMIT}) {{
                          nodes {{
                            name
                          }}
                        }}
                        number
                        title
                        url
                        state
                        closedAt
                        createdAt
                        updatedAt
                        author {{
                          login
                        }}
                      }}
                    }}
                  }}
                }}
              }}
            }}
            """)
        result = await runQuery(overviewQuery)
        prs += result['repository']['pullRequests']['nodes']
        cursor = result['repository']['pullRequests']['pageInfo']['endCursor']
        # cursor = None # Comment out this line to fetch all
    prs = list(map(clean_PR_data, prs))
    prsDF = pd.DataFrame(prs)
    prsDF.to_csv('prs_data.csv', index=False)
    return prsDF

await get_PR_data()


Fetching PRs, this can take a while...


,title,author,assignees,number,url,state,closedAt,createdAt,updatedAt,labels,issues,hoursFromOpen,hoursToClose
0,Update Kanban doc to cover holes,bmcilw1,{bmcilw1},2125,https://github.com/weiks/esports-backend/pull/...,OPEN,NaT,2022-09-02 16:58:07+00:00,2022-09-05T20:24:02Z,"{docs, 0-trigger-ci}",[],75.506075,NaN
1,[Doc] added yunite last steps,brianmazzocchi,{brianmazzocchi},2142,https://github.com/weiks/esports-backend/pull/...,MERGED,2022-09-05 20:10:19+00:00,2022-09-05 19:37:52+00:00,2022-09-05T20:10:19Z,{docs},[],0.843576,0.540833
2,[WIP] Add reminders toggle endpoint,Mathspy,{Mathspy},2136,https://github.com/weiks/esports-backend/pull/...,OPEN,NaT,2022-09-05 15:32:34+00:00,2022-09-05T19:46:30Z,{0-trigger-ci},[],4.931909,NaN
3,Update Invite teammates link generation not to...,mikiasyonas,{mikiasyonas},2141,https://github.com/weiks/esports-backend/pull/...,CLOSED,2022-09-05 19:03:22+00:00,2022-09-05 18:43:08+00:00,2022-09-05T19:03:22Z,"{bug, critical, triage-me}","[{'assignees': {'mikiasyonas'}, 'labels': {'bu...",1.755798,0.337222
4,add more info to unexpected errors,gonzalovelasco,{gonzalovelasco},2135,https://github.com/weiks/esports-backend/pull/...,MERGED,2022-09-05 15:01:02+00:00,2022-09-05 14:39:46+00:00,2022-09-05T15:01:03Z,"{bug, critical}","[{'assignees': {}, 'labels': {'bug', 'critical...",5.811909,0.354444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655,Add auth middleware to discord/token endpoint,Mathspy,{Mathspy},6,https://github.com/weiks/esports-backend/pull/6,MERGED,2020-03-18 01:17:28+00:00,2020-03-18 01:16:02+00:00,2020-03-18T01:17:34Z,{},[],21643.207538,0.023889
1656,Add users to db on successful OAuth,Mathspy,{Mathspy},5,https://github.com/weiks/esports-backend/pull/5,MERGED,2020-03-17 22:39:52+00:00,2020-03-17 22:33:44+00:00,2020-03-17T22:39:55Z,{},[],21645.912538,0.102222
1657,Swapped REDIRECT_URI to be env var based,Mathspy,{Mathspy},4,https://github.com/weiks/esports-backend/pull/4,MERGED,2020-03-14 21:08:57+00:00,2020-03-14 21:00:49+00:00,2020-03-14T21:09:00Z,{},[],21719.461149,0.135556
1658,Added new slot fields required by reworked qBonus,Mathspy,{Mathspy},2,https://github.com/weiks/esports-backend/pull/2,MERGED,2020-02-15 15:44:36+00:00,2020-02-15 15:27:31+00:00,2020-02-18T00:27:06Z,{},[],22397.016149,0.284722


## Get Issue Data

In [3]:
async def get_issue_data():
    cursor = ""
    issues = []
    print('Fetching issues, this can take a while...')

    while cursor != None:
        overviewQuery = gql(
            f"""
            query getRepoData {{
              repository(owner: "{REPO_OWNER}", name: "{REPO_NAME}") {{
                issues(
                  first: {MAX_WEEK_ISSUES}, 
                  orderBy: {{ field: UPDATED_AT, direction: DESC }}
                  { f'after: "{cursor}"' if cursor else '' }
                ) {{
                  pageInfo {{
                    hasNextPage
                    endCursor
                  }}
                  nodes {{
                    title
                    author {{
                      login
                    }}
                    assignees(first: 10) {{
                      nodes {{
                        login
                      }}
                    }}
                    number
                    url
                    state
                    closedAt
                    createdAt
                    updatedAt
                    labels(first: {MAX_LABEL_LIMIT}) {{
                      nodes {{
                        name
                      }}
                    }}
                  }}
                }}
              }}
            }}
            """)
        result = await runQuery(overviewQuery)
        issues += result['repository']['issues']['nodes']
        cursor = result['repository']['issues']['pageInfo']['endCursor']
        # cursor = None # Comment out this line to fetch all
    issues = list(map(clean_issue_data, issues))
    issuesDF = pd.DataFrame(issues)
    issuesDF.to_csv('issues_data.csv', index=False)
    return issuesDF


await get_issue_data()


Fetching issues, this can take a while...


,title,author,assignees,number,url,state,closedAt,createdAt,updatedAt,labels,daysFromOpen,daysToClose
0,[Milestone] Create tests for Business layer DAO,CarlosViller,{},2139,https://github.com/weiks/esports-backend/issue...,OPEN,NaT,2022-09-05 16:14:50+00:00,2022-09-05T19:47:25Z,{enhancement},0.176657,NaN
1,[Milestone] Refactor and cleanup,CarlosViller,{},2138,https://github.com/weiks/esports-backend/issue...,OPEN,NaT,2022-09-05 16:07:57+00:00,2022-09-05T18:49:41Z,{enhancement},0.181437,NaN
2,Migrate the `reminders` feature to the new qBot,alexangc,"{gonzalovelasco, Mathspy}",2131,https://github.com/weiks/esports-backend/issue...,OPEN,NaT,2022-09-05 10:25:15+00:00,2022-09-05T18:39:39Z,{},0.419423,NaN
3,[Bug] Challonge get results failed,ajcooper99,{},2126,https://github.com/weiks/esports-backend/issue...,OPEN,NaT,2022-09-02 18:13:14+00:00,2022-09-05T18:39:30Z,"{bug, critical}",3.094435,NaN
4,[Bug] Error to join an event with an invalid i...,gonzalovelasco,{mikiasyonas},2108,https://github.com/weiks/esports-backend/issue...,OPEN,NaT,2022-09-01 20:44:12+00:00,2022-09-05T18:35:20Z,"{bug, critical, triage-me}",3.989597,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
477,[Airbrake] [Production] 10 ABORTED: The refere...,weiks,{Mathspy},527,https://github.com/weiks/esports-backend/issue...,CLOSED,2021-03-31 14:43:40+00:00,2021-03-11 08:16:28+00:00,2021-03-31T14:43:40Z,{},543.508857,20.268889
478,[Airbrake] [Production] Cannot read property '...,weiks,{},392,https://github.com/weiks/esports-backend/issue...,CLOSED,2021-01-23 20:47:12+00:00,2021-01-23 17:12:17+00:00,2021-01-23T20:47:12Z,{},590.136762,0.149248
479,"[Airbrake] [Production] Value for argument ""do...",weiks,{},391,https://github.com/weiks/esports-backend/issue...,CLOSED,2021-01-23 20:38:37+00:00,2021-01-23 16:48:08+00:00,2021-01-23T20:38:37Z,{},590.153533,0.160058
480,[Airbrake] [Production] Channel closed by serv...,weiks,{},334,https://github.com/weiks/esports-backend/issue...,CLOSED,2020-12-29 19:42:18+00:00,2020-12-29 09:12:59+00:00,2020-12-29T19:42:18Z,{},615.469609,0.437025
